Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [ ]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2023-02-06 10:06:54--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2023-02-06 10:06:54--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.08s   

2023-02-06 10:06:55 (33.2 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/

In [ ]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')
train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-14.png', 'happy2-08.png', 'happy2-16.png', 'happy2-13.png', 'happy2-11.png', 'happy1-10.png', 'happy2-05.png', 'happy1-03.png', 'happy2-19.png', 'happy2-15.png']
['sad1-19.png', 'sad1-08.png', 'sad2-00.png', 'sad1-12.png', 'sad1-03.png', 'sad2-05.png', 'sad1-07.png', 'sad1-17.png', 'sad1-18.png', 'sad1-09.png']


In [ ]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-14.png', 'happy2-08.png', 'happy2-16.png', 'happy2-13.png', 'happy2-11.png', 'happy1-10.png', 'happy2-05.png', 'happy1-03.png', 'happy2-19.png', 'happy2-15.png']
['sad1-19.png', 'sad1-08.png', 'sad2-00.png', 'sad1-12.png', 'sad1-03.png', 'sad2-05.png', 'sad1-07.png', 'sad1-17.png', 'sad1-18.png', 'sad1-09.png']


In [ ]:

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.999:
            print("\nДостигнута точность 99.9%, обучение прерывается!")
            self.model.stop_training = True

callbacks = myCallback()

# Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

  # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
  # и train_generator, вызовом train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# Используйте target_size  150 X 150.
rain_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

# model fitting
history = model.fit(
    train_generator,
    steps_per_epoch=4,  # 80 изображений = batch_size * steps_per_epoch
    epochs=15,          # Количество эпох для обучения
    verbose=1,
    callbacks=[callbacks]
)

# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"
print(history.epoch, history.history['accuracy'][-1])